In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
#os.chdir('..')

In [ ]:
data_config = 1
data_dir = f"datasets/type_{data_config}"
results_dir = "results_relu_exhaustive/slow"
model_names = ["Dirichlet Horseshoe"] #["Regularized Horseshoe", "Dirichlet Horseshoe", "Gaussian"]

all_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"type_{data_config}/{base_config_name}"  # → "config_1/GAM_N100_p8_sigma1.00_seed1"
    print(full_config_path)
    print(base_config_name)
    fits = get_model_fits(
        config=full_config_path,
        results_dir=results_dir,
        models=model_names,
        include_prior=False,
    )

    all_fits[base_config_name] = fits  # use clean key



In [3]:
def get_N_sigma(seed):
    N = 100 if seed in [1, 2, 3, 4] else 200
    sigma = 1.0 if seed in [1, 2, 7, 8] else 3.0
    if seed == 19:
        N, sigma = 1000, 1.0
    return N, sigma

def compute_rmse_results(seeds, models, all_fits, get_N_sigma):
    results = []
    posterior_means = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed)
        dataset_key = f'GAM_N{N}_p8_sigma{sigma:.2f}_seed{seed}'
        path = f"datasets/type_{data_config}/{dataset_key}.npz"

        try:
            data = np.load(path)
            y_test = data["y_test"]
        except FileNotFoundError:
            print(f"[SKIP] File not found: {path}")
            continue

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                output_test = fit.stan_variable("output_test")  # (S, N/5, O)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = output_test.shape[0]
            rmses = np.zeros(S)

            for i in range(S):
                y_hat = output_test[i]
                rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test.squeeze()) ** 2))
                
            posterior_mean = np.mean(output_test, axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean.squeeze() - y_test.squeeze()) ** 2))

            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'posterior_mean_rmse': posterior_mean_rmse
            })

            for i in range(S):
                results.append({
                    'seed': seed,
                    'N': N,
                    'sigma': sigma,
                    'model': model,
                    'rmse': rmses[i]
                })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse


In [4]:
seeds = [1, 2, 3, 4, 7, 8, 9, 10]#, 19]

df_rmse, df_posterior_rmse = compute_rmse_results(
    seeds, model_names, all_fits, get_N_sigma
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

for i, N_val in enumerate([100, 200]):
    ax = axes[i]
    sns.boxplot(
        data=df_rmse[df_rmse['N'] == N_val],
        x="model", y="rmse", hue="sigma", ax=ax
    )
    sns.scatterplot(
        data=df_posterior_rmse[df_posterior_rmse['N'] == N_val],
        x="model", y="posterior_mean_rmse", hue="sigma",
        marker="X", s=100, ax=ax, zorder=10, legend=False
    )
    ax.set_title(f"RMSE Distribution (N = {N_val})")
    ax.set_xlabel("Model")
    #ax.set_ylim(0, 5)
    ax.set_ylabel("RMSE")
    ax.grid(True)

axes[1].legend(title="Sigma")
plt.tight_layout()
#plt.savefig(f"figures/GAM_{data_config}/rmse_models.png", dpi=300, bbox_inches='tight')
plt.show()

In [6]:
from properscoring import crps_ensemble
import pandas as pd
import numpy as np

def compute_crps_results(seeds, models, all_fits, get_N_sigma):
    results = []
    posterior_means = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed)
        dataset_key = f'GAM_N{N}_p8_sigma{sigma:.2f}_seed{seed}'
        path = f"datasets/type_{data_config}/{dataset_key}.npz"

        try:
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # (N_test,)
        except FileNotFoundError:
            print(f"[SKIP] File not found: {path}")
            continue

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                output_test = fit.stan_variable("output_test").squeeze()  # shape: (S, N_test)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            # Compute CRPS for each test point using all samples
            crps_vals = crps_ensemble(y_test, output_test.T)  # shape: (N_test,)
            mean_crps = np.mean(crps_vals)

            # Store average CRPS
            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'mean_crps': mean_crps
            })

            # Store CRPS per observation (optional granularity)
            for i in range(len(y_test)):
                results.append({
                    'seed': seed,
                    'N': N,
                    'sigma': sigma,
                    'model': model,
                    'obs': i,
                    'crps': crps_vals[i]
                })

    df_crps = pd.DataFrame(results)
    df_crps_summary = pd.DataFrame(posterior_means)

    return df_crps, df_crps_summary


In [7]:
seeds = [1, 2, 3, 4, 7, 8, 9, 10]#, 19]

df_crps, df_crps_summary = compute_crps_results(
    seeds, model_names, all_fits, get_N_sigma
)

In [ ]:
df_crps

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

for i, N_val in enumerate([100, 200]):
    ax = axes[i]
    sns.boxplot(
        data=df_crps[df_crps['N'] == N_val],
        x="model", y="crps", hue="sigma", ax=ax
    )
    ax.set_title(f"CRPS Distribution (N = {N_val})")
    ax.set_xlabel("Model")
    #ax.set_ylim(0, 5)
    ax.set_ylabel("CRPS")
    ax.grid(True)

axes[1].legend(title="Sigma")
plt.tight_layout()
#plt.savefig(f"figures/GAM_{data_config}/rmse_models.png", dpi=300, bbox_inches='tight')
plt.show()